In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common import results_plotter
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.monitor import Monitor




# PPO

## Create Callback

In [10]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf


    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), "timesteps")
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(
                        f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}"
                    )

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}.zip")
                    self.model.save(self.save_path)

        return True

## Plot Functions

In [11]:
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, "valid")


def plot_results(log_folder, title="Learning Curve"):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), "timesteps")
    #y = moving_average(y, window=50)
    # Truncate x
    #x = x[len(x) - len(y) :]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel("Number of Timesteps")
    plt.ylabel("Rewards")
    plt.title(title + " Smoothed")
    plt.show()

## Create Gif

In [23]:
import imageio
import numpy as np

def create_gif(env, model_path, path, name):
    model = PPO.load(model_path, env=env)
    images = []

    vec_env = model.get_env()
    obs = vec_env.reset()
    img = vec_env.render()
    for i in range(500):
        images.append(img)
        action, _ = model.predict(obs)
        obs, _, _ ,_ = vec_env.step(action)
        img = vec_env.render(mode='rgb_array')
    gif_name =  path + name + '.gif'
    imageio.mimsave(gif_name, [np.array(img) for i, img in enumerate(images) if i%2 == 0], duration=120)

## Normal parameters

In [13]:
# Create environment
env = gym.make("CarRacing-v2")

In [14]:
#setup callback

# Create log dir
log_dir = "ppo_normal/"
os.makedirs(log_dir, exist_ok=True)

# Wrap the environment
env = Monitor(env, log_dir)

callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

### Train agent

In [15]:
# Instantiate the agent
model = PPO("MlpPolicy", env, verbose=1)
# Train the agent and display a progress bar
timesteps = 25000
model.learn(total_timesteps=int(timesteps), callback=callback)


Using cpu device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Num timesteps: 1000
Best mean reward: -inf - Last mean reward per episode: -60.73
Saving new best model to ppo_normal/best_model.zip
Num timesteps: 2000
Best mean reward: -60.73 - Last mean reward per episode: -57.07
Saving new best model to ppo_normal/best_model.zip
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -57.1    |
| time/              |          |
|    fps             | 33       |
|    iterations      | 1        |
|    time_elapsed    | 60       |
|    total_timesteps | 2048     |
---------------------------------
Num timesteps: 3000
Best mean reward: -57.07 - Last mean reward per episode: -54.19
Saving new best model to ppo_normal/best_model.zip
Num timesteps: 4000
Best mean reward: -54.19 - Last mean reward per episode: -54.71
------------------------------------------
| rollout/                |          

In [16]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=5)

In [17]:
print(mean_reward, std_reward)

-77.0105234 7.538367439023164


In [18]:
# Helper from the library
#results_plotter.plot_results(
 #   [log_dir], timesteps, results_plotter.X_TIMESTEPS, "PPO CarRacing-v2"
#)

In [19]:
#plot_results(log_dir)

### Load pre-trained model

In [20]:
test_env = gym.make("CarRacing-v2", render_mode='human')
dir = os.path.join(log_dir, "best_model.zip")
model = PPO.load(dir, env=test_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [21]:
vec_env = model.get_env()
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = vec_env.step(action)
    #obs, rewards, dones, info = test_env.step(action)
    vec_env.render()
    if done:
        break

vec_env.close()

In [24]:
test_env = gym.make("CarRacing-v2", render_mode='rgb_array')
dir = os.path.join(log_dir, "best_model")
create_gif(test_env, dir, log_dir, "ppo_car")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


## Discrete space

In [25]:
# Create environment
env = gym.make("CarRacing-v2", continuous=False)

In [26]:
#setup callback

# Create log dir
log_dir = "ppo_discrete/"
os.makedirs(log_dir, exist_ok=True)

# Wrap the environment
env = Monitor(env, log_dir)

callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

### Train agent

In [27]:
# Instantiate the agent
model = PPO("MlpPolicy", env, verbose=1)
# Train the agent and display a progress bar
timesteps = 25000
model.learn(total_timesteps=int(timesteps), callback=callback)

Using cpu device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Num timesteps: 1000
Best mean reward: -inf - Last mean reward per episode: -61.65
Saving new best model to ppo_discrete/best_model.zip
Num timesteps: 2000
Best mean reward: -61.65 - Last mean reward per episode: -56.35
Saving new best model to ppo_discrete/best_model.zip
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -56.4    |
| time/              |          |
|    fps             | 40       |
|    iterations      | 1        |
|    time_elapsed    | 50       |
|    total_timesteps | 2048     |
---------------------------------
Num timesteps: 3000
Best mean reward: -56.35 - Last mean reward per episode: -53.29
Saving new best model to ppo_discrete/best_model.zip
Num timesteps: 4000
Best mean reward: -53.29 - Last mean reward per episode: -47.80
Saving new best model to ppo_discrete/best_model.zip
---------------------

In [28]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=5)

In [29]:
print(mean_reward, std_reward)

-51.498120199999995 35.88498520079649


In [ ]:
# Helper from the library
#results_plotter.plot_results(
 #   [log_dir], timesteps, results_plotter.X_TIMESTEPS, "PPO CarRacing-v2"
#)

In [ ]:
#plot_results(log_dir)

### Load pre-trained model

In [30]:
test_env = gym.make("CarRacing-v2", render_mode='human', continuous=False)
dir = os.path.join(log_dir, "best_model.zip")
model = PPO.load(dir, env=test_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [31]:
vec_env = model.get_env()
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = vec_env.step(action)
    #obs, rewards, dones, info = test_env.step(action)
    vec_env.render()
    if done:
        break

vec_env.close()

In [32]:
test_env = gym.make("CarRacing-v2", render_mode='rgb_array', continuous=False)
dir = os.path.join(log_dir, "best_model")
create_gif(test_env, dir, log_dir, "ppo_car")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


## Lap completion percentage change 

In [33]:
# Create environment
env = gym.make("CarRacing-v2", lap_complete_percent=0.5)

In [34]:
#setup callback

# Create log dir
log_dir = "ppo_lap/"
os.makedirs(log_dir, exist_ok=True)

# Wrap the environment
env = Monitor(env, log_dir)

callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

### Train agent

In [36]:
# Instantiate the agent
model = PPO("MlpPolicy", env, verbose=1)
# Train the agent and display a progress bar
timesteps = 25000
model.learn(total_timesteps=int(timesteps), callback=callback)

Using cpu device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Num timesteps: 1000
Best mean reward: -inf - Last mean reward per episode: -52.19
Saving new best model to ppo_lap/best_model.zip
Num timesteps: 2000
Best mean reward: -52.19 - Last mean reward per episode: -54.20
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -54.2    |
| time/              |          |
|    fps             | 41       |
|    iterations      | 1        |
|    time_elapsed    | 49       |
|    total_timesteps | 2048     |
---------------------------------
Num timesteps: 3000
Best mean reward: -52.19 - Last mean reward per episode: -50.00
Saving new best model to ppo_lap/best_model.zip
Num timesteps: 4000
Best mean reward: -50.00 - Last mean reward per episode: -48.77
Saving new best model to ppo_lap/best_model.zip
------------------------------------------
| rollout/                |              |
|  

In [37]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=5)

In [38]:
print(mean_reward, std_reward)

-69.51982840000001 1.939266231595713


In [ ]:
# Helper from the library
#results_plotter.plot_results(
 #   [log_dir], timesteps, results_plotter.X_TIMESTEPS, "PPO CarRacing-v2"
#)

In [ ]:
#plot_results(log_dir)

### Load pre-trained model

In [40]:
test_env = gym.make("CarRacing-v2", render_mode='human')#, continuous=False)
dir = os.path.join(log_dir, "best_model.zip")
model = PPO.load(dir, env=test_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [ ]:
vec_env = model.get_env()
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = vec_env.step(action)
    #obs, rewards, dones, info = test_env.step(action)
    vec_env.render()
    if done:
        break

vec_env.close()

In [43]:
test_env = gym.make("CarRacing-v2", render_mode='rgb_array')
dir = os.path.join(log_dir, "best_model")
create_gif(test_env, dir, log_dir, "ppo_car")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


## Train with CNN instead of MLP

In [44]:
env = gym.make("CarRacing-v2", render_mode = "rgb_array")

In [45]:
#setup callback

# Create log dir
log_dir = "ppo_cnn/"
os.makedirs(log_dir, exist_ok=True)

# Wrap the environment
env = Monitor(env, log_dir)

callback = SaveOnBestTrainingRewardCallback(check_freq=1000, log_dir=log_dir)

In [46]:
# Instantiate the agent
model = PPO("CnnPolicy", env, verbose=1)
# Train the agent and display a progress bar
timesteps = 25000
model.learn(total_timesteps=int(timesteps), callback=callback)

Using cpu device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Num timesteps: 1000
Best mean reward: -inf - Last mean reward per episode: -55.93
Saving new best model to ppo_cnn/best_model.zip
Num timesteps: 2000
Best mean reward: -55.93 - Last mean reward per episode: -58.79
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -58.8    |
| time/              |          |
|    fps             | 42       |
|    iterations      | 1        |
|    time_elapsed    | 47       |
|    total_timesteps | 2048     |
---------------------------------
Num timesteps: 3000
Best mean reward: -55.93 - Last mean reward per episode: -56.49
Num timesteps: 4000
Best mean reward: -55.93 - Last mean reward per episode: -54.53
Saving new best model to ppo_cnn/best_model.zip
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_

In [47]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=5)

print(mean_reward, std_reward)

-51.159724 24.48586646563704


In [ ]:
# Helper from the library
#results_plotter.plot_results(
 #   [log_dir], timesteps, results_plotter.X_TIMESTEPS, "PPO CarRacing-v2"
#)

In [ ]:
#plot_results(log_dir)

## Load pre-trained model

In [48]:
test_env = gym.make("CarRacing-v2", render_mode='human')
dir = os.path.join(log_dir, "best_model.zip")
model = PPO.load(dir, env=test_env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [49]:
vec_env = model.get_env()
obs = vec_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = vec_env.step(action)
    #obs, rewards, dones, info = test_env.step(action)
    vec_env.render()
    if done:
        break

vec_env.close()

In [50]:
test_env = gym.make("CarRacing-v2", render_mode='rgb_array')
dir = os.path.join(log_dir, "best_model")
create_gif(test_env, dir, log_dir, "ppo_car")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
